In [1]:
from __future__ import print_function

import datetime
import math
import json
import six
import webbrowser

from ortools.constraint_solver import pywrapcp
from draw_map import draw_paths

In [2]:
AVERAGE_SPEED_MPH = 50000

In [3]:
def get_total_seconds(time_slot):
    if time_slot is None:
        return None
    if isinstance(time_slot, six.string_types):
        time_slot = datetime.datetime.strptime(time_slot, '%H:%M:%S')

    return (
        time_slot.hour * 3600 + time_slot.minute * 60 + time_slot.second)

In [4]:
def crow_fly_distance(origin_location, destination_location):
    # Radius of the earth in meters.
    earth_radius = 6371 * 1000
    # Formula source : http://www.movable-type.co.uk/scripts/latlong.html.
    delta_lat = math.radians(
        float(destination_location['latitude']) - float(origin_location['latitude'])
    )
    delta_lng = math.radians(
        float(destination_location['longitude']) - float(origin_location['longitude'])
    )

    a = (
        math.sin(delta_lat / 2.0) * math.sin(delta_lat / 2.0) +
        math.cos(math.radians(origin_location['latitude'])) *
        math.cos(math.radians(destination_location['latitude'])) *
        math.sin(delta_lng / 2.0) * math.sin(delta_lng / 2.0))
    return earth_radius * 2.0 * math.atan2(
        math.sqrt(a), math.sqrt(1.0 - a))

In [5]:
def crow_fly_duration(origin_location, destination_location):
    return int(crow_fly_distance(
        origin_location, destination_location) / AVERAGE_SPEED_MPH * 3600)

In [6]:
class OptimizationModel(object):
    def __init__(self, visits, start_indexes, end_indexes):
        self.visits = visits
        self.start_indexes = start_indexes
        self.end_indexes = end_indexes
        self.index_to_visit = dict(
            list(enumerate(visits))
        )

        for start_index in self.start_indexes:
            assert 0 <= start_index < len(visits)

        assert len(self.start_indexes) == len(set(self.start_indexes))
        assert len(self.end_indexes) == len(set(self.end_indexes))
        assert len(self.start_indexes) == len(self.end_indexes)

        self.number_of_vehicles = len(self.start_indexes)

        self.routing_model = pywrapcp.RoutingModel(
            len(self.visits), self.number_of_vehicles,
            self.start_indexes,
            self.end_indexes
        )
        self.solution = None
        self.time_dimension = None
        self.day_duraiton = 16 * 3600  # 16 hours

    def solve(self):
        if self.solution is not None:
            print('Solution already found, use show() to see the results')
            return

        parameters = self.routing_model.DefaultSearchParameters()
        parameters.log_search = True
        distance_function = self.distance_function_callback()
        self.routing_model.SetArcCostEvaluatorOfAllVehicles(
            distance_function
        )

        solution = self.routing_model.SolveWithParameters(parameters)

        if not solution:
            print('No solution :(')
            return
        self.solution = solution
        print(
            'Found solution! Total distance: {0} km'.format(
                self.solution.ObjectiveValue() / 100 / 1000
            )
        )

    def show(self):
        if self.solution is None:
            print('Solution not found yet')
            return

        all_paths = []

        for vehicle_number in range(self.number_of_vehicles):
            vehicle_route = self.get_route_of_vehicle(vehicle_number)
            all_paths.append([
                self.serialize_visit_for_map(visit_index)
                for visit_index in vehicle_route
            ])

        center = self.serialize_visit_for_map(self.start_indexes[0])

        draw_paths(
            center=[center[0], center[1]],
            paths=all_paths,
            file_name='output.html'
        )
        return 'output.html'

    def get_route_of_vehicle(self, vehicle_number):
        route = []
        current_index = self.routing_model.Start(vehicle_number)
        while not self.routing_model.IsEnd(current_index):
            route.append(
                self.routing_model.IndexToNode(current_index)
            )
            current_index = self.solution.Value(
                self.routing_model.NextVar(current_index)
            )
        route.append(self.routing_model.IndexToNode(current_index))
        return route

    def convert_index_route_to_visit_route(self, route):
        return [
            self.visits[index]
            for index in route
        ]

    def serialize_visit_for_map(self, index):
        visit = self.index_to_visit[index]
        return (
            visit['coordinates']['latitude'], visit['coordinates']['longitude'],
            visit['id']
        )

    def get_node_location(self, node):
        return self.index_to_visit[node]['coordinates']

    def get_node_duration(self, node):
        return int(self.index_to_visit[node]['duration'] * 3600)

    def distance_function_callback(self):
        node_to_location = {
            node: self.get_node_location(node)
            for node in range(self.routing_model.nodes())
        }

        def distance_callback(from_node, to_node):
            from_location = node_to_location[from_node]
            to_location = node_to_location[to_node]
            return int(100 * crow_fly_distance(from_location, to_location))

        return distance_callback

    def time_function_callback(self):
        node_to_location = {
            node: self.get_node_location(node)
            for node in range(self.routing_model.nodes())
        }
        node_to_duration = {
            node: self.get_node_duration(node)
            for node in range(self.routing_model.nodes())
        }

        def time_function(from_node, to_node):
            from_duration = node_to_duration[from_node]
            from_location = node_to_location[from_node]
            to_location = node_to_location[to_node]

            return (
                from_duration +
                crow_fly_duration(from_location, to_location)
            )
        return time_function

In [7]:
with open('places.json') as f:
    interesting_places = json.load(f)

work_nodes = [
    {
        'id': 'Sankt Oberholz',
        'constraints': {
            'time_window_start': '7:00:00',
            'time_window_end': '11:00:00',
        },
        'coordinates': {
            'latitude': 52.529763,
            'longitude': 13.407482
        },
        'type': 'work',
        'duration': 4
    },
    {
        'id': 'Sankt Oberholz',
        'constraints': {
            'time_window_start': '11:00:00',
        },
        'coordinates': {
            'latitude': 52.529763,
            'longitude': 13.407482
        },
        'type': 'work',
        'duration': 4
    },
]

hotel_node_start = {
    'id': 'Mercure Hotel',
    'constraints': {
        'time_window_end': '10:00:00',
    },
    'coordinates': {
        'latitude': 52.524388,
        'longitude': 13.420468
    },
    'type': 'rest',
    'duration': 8
}

hotel_node_end = {
    'id': 'Mercure Hotel',
    'constraints': {
        'time_window_end': '10:00:00',
    },
    'coordinates': {
        'latitude': 52.524388,
        'longitude': 13.420468
    },
    'type': 'rest',
    'duration': 0
}

number_of_days = 1

hotel_node_starts = [hotel_node_start] * number_of_days
hotel_node_ends = [hotel_node_end] * number_of_days

all_visits = interesting_places + work_nodes
start_indexes = list(
    range(
        len(all_visits),
        len(all_visits) + number_of_days
    )
)
all_visits += hotel_node_starts
end_indexes = list(
    range(
        len(all_visits),
        len(all_visits) + number_of_days
    )
)
all_visits += hotel_node_ends

In [8]:
model = OptimizationModel(
    all_visits, start_indexes=start_indexes, end_indexes=end_indexes
)

In [9]:
model.solve()

Found solution! Total distance: 13 km


In [10]:
webbrowser.open_new_tab(model.show())

True